In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done

# All requested packages already installed.

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done

# All requested packages already installed.

Libraries imported.


### New York City Dataset

In [3]:
#Reading New York Neighborhood data file into data frame

path='/Users/ShivaNangi/Downloads/NHoodNameCentroids.csv'
nyc_df=pd.read_csv(path)
nyc_df.head()

,the_geom,OBJECTID,Name,Stacked,AnnoLine1,AnnoLine2,AnnoLine3,AnnoAngle,Borough
0,POINT (-73.8472005205491 40.89470517661004),1,Wakefield,1,Wakefield,NaN,NaN,0,Bronx
1,POINT (-73.82993910812405 40.87429419303015),2,Co-op City,2,Co-op,City,NaN,0,Bronx
2,POINT (-73.82780644716419 40.88755567735082),3,Eastchester,1,Eastchester,NaN,NaN,0,Bronx
3,POINT (-73.90564259591689 40.895437426903875),4,Fieldston,1,Fieldston,NaN,NaN,0,Bronx
4,POINT (-73.91258546108577 40.89083449389134),5,Riverdale,1,Riverdale,NaN,NaN,0,Bronx


In [4]:
# Extracting only required coloumns from the main dataset

nyc_data=nyc_df[['Name','Borough']]
nyc_data.head()

,Name,Borough
0,Wakefield,Bronx
1,Co-op City,Bronx
2,Eastchester,Bronx
3,Fieldston,Bronx
4,Riverdale,Bronx


In [5]:
# Cleaning up dataset to get data about latitudes and longitudes of each neighborhood

lat_lon=nyc_df[['the_geom']]
lat_lon['the_geom'] = lat_lon['the_geom'].str.strip('POINT (')


lat_lon[['Longitude','Latitude']] = lat_lon.the_geom.str.split(" ",expand=True,)


lat_lon['Latitude'] = lat_lon['Latitude'].str.strip(')')
lat_lon=lat_lon[['Latitude','Longitude']]


/Users/ShivaNangi/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/ShivaNangi/anaconda/lib/python3.6/site-packages/pandas/core/frame.py:2440: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
/Users/ShivaNangi/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http:

In [6]:
#Concatenating all the dataframes to form a final dataframe

nyc_final_df = pd.concat([nyc_data,lat_lon], axis=1, ignore_index=True)
nyc_final_df.head()

,0,1,2,3
0,Wakefield,Bronx,40.89470517661004,-73.8472005205491
1,Co-op City,Bronx,40.87429419303015,-73.82993910812405
2,Eastchester,Bronx,40.88755567735082,-73.82780644716419
3,Fieldston,Bronx,40.895437426903875,-73.90564259591689
4,Riverdale,Bronx,40.89083449389134,-73.91258546108577


In [7]:
#Assigning headers

nyc_final_df.columns=['Neighborhood','Borough','Latitude','Longitude']

In [8]:
#Rearraging columns
#the final data frame that we will working will be nyc_df

nyc_df=nyc_final_df[['Borough','Neighborhood','Latitude','Longitude']]
nyc_df.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.89470517661004,-73.8472005205491
1,Bronx,Co-op City,40.87429419303015,-73.82993910812405
2,Bronx,Eastchester,40.88755567735082,-73.82780644716419
3,Bronx,Fieldston,40.895437426903875,-73.90564259591689
4,Bronx,Riverdale,40.89083449389134,-73.91258546108577


In [9]:
nyc_df.dtypes

Borough         object
Neighborhood    object
Latitude        object
Longitude       object
dtype: object

In [10]:
# using dictionary to convert specific columns 
convert_dict = {'Latitude': float,
                'Longitude': float
               }  
  
nyc_df = nyc_df.astype(convert_dict) 
print(nyc_df.dtypes)

Borough          object
Neighborhood     object
Latitude        float64
Longitude       float64
dtype: object


### Toronto Data set

#### Importing all the required libraries for web scraping

In [11]:
# import libraries
from urllib.request import urlopen as UReq
from bs4 import BeautifulSoup as soup
# specify the url
urlpage = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' 


# query the website and return the html to the variable 'page'
UClient = UReq(urlpage)

# Opening up the connection and grabbing the page
page_html=UClient.read()


### Load UClient, turn into soup and get the tables

In [12]:
# parse the html using beautiful soup and store in variable 'page_soup'

page_soup = soup(page_html, 'html.parser')

#Scrape out the required table 

tables2=page_soup.find_all('table',class_="wikitable sortable")


In [ ]:
#Using prettify to get the html code of the required field
print(page_soup.prettify())

## Exatracting information from web using Html tags

In [14]:
#Exacted headers for the tables
header=[]
for headers in tables2:
    ths=headers.findAll('th')
    for th in ths:
        header.append(th.text)
header

['Postcode', 'Borough', 'Neighbourhood\n']

In [ ]:
#Exacting required text fields from the wiki table 
rowValList = []

for i in range(len(page_soup.findAll('td'))):
    rowVal = page_soup.findAll('td')[i].get_text()
    rowValList.append(rowVal)
rowValList

In [16]:
#separating the text from the above list into seperate lists
postcode = []
for i in range(0, len(rowValList), 3):
    postcode.append(rowValList[i])
    
borough = []
for i in range(1, len(rowValList), 3):
    borough.append(rowValList[i])
    
neigh = []
for i in range(2, len(rowValList), 3):
    neigh.append(rowValList[i])


### Cleaning up the data to match the requirement

In [17]:
#cleaning the data and creating a data frame for each list above
pc_df = pd.DataFrame(postcode)
pc_df.drop(pc_df.tail(12).index,inplace=True)
#print(pc_df)


b_df = pd.DataFrame(borough)
b_df.drop(b_df.tail(11).index,inplace=True)
#b_df

n_df = pd.DataFrame(neigh)#,columns=neigh)
n_df.drop(n_df.tail(11).index,inplace=True)
#n_df

In [18]:
#concatenating all the dataframes column wise into one dataframe called result
final1 = pd.concat([pc_df, b_df,n_df], axis=1, ignore_index=True)


In [19]:
#adding headers to the dataframe
final1.columns=['Postalcode', 'Borough', 'Neighbourhood']


In [20]:
#Striping out unnecesary text from neighbourhood field and the final dataframe is ready to use

final1['Neighbourhood'] = final1['Neighbourhood'].str.rstrip('\n')



In [21]:
#Just duplicating the dataframe before further cleaning so that the original dataframe will not be effected

final2=final1.copy()


### Processing only rows with assigned Borough

In [22]:
# Get names of indexes for which column Neighbourhood has value Not assigned
indexNames = final1[final1['Borough'] == 'Not assigned' ].index
indexNames 
# Delete these row indexes from dataFrame
final1.drop(indexNames , inplace=True)
final1=final1.reset_index(drop=True)


### Grouping the dataset with PostalCode and Boroughv

In [23]:
#Grouping more than one neighbourhood with same postal codes and Borough
final1 = final1.groupby(['Postalcode','Borough']).agg(lambda x: ", ".join(x.astype(str))).reset_index()
final1.head()

,Postalcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Assigning the 'Not assigned' values in Neighbourhood with the value in Borough

In [ ]:
final1.loc[final1.Neighbourhood == 'Not assigned', 'Neighbourhood'] = final1.Borough
final1

In [25]:
#Renaming column name 

final1.rename(columns = {'Postalcode':'Postal Code'}, inplace = True) 
final1.columns

Index(['Postal Code', 'Borough', 'Neighbourhood'], dtype='object')

#### Shape of the data frame

In [26]:
#Final number or rows and columns in the data frame

final1.shape

(103, 3)

## Getting latitudes and longitudes for each neighbourhood

In [27]:
!wget -q -O 'Geospatial_data.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

/bin/sh: wget: command not found
Data downloaded!


In [28]:
#reading CSV file which contains latitudes and longitudes
df_geo = pd.read_csv(r'/Users/ShivaNangi/Downloads/Geospatial_Coordinates.csv')


### Merging data frame which contain information about pin code,borough and neighborhood and data frame which contains latitudes and longitudes data to make a final data frame which is used to get some insights

In [29]:
df_lat_long = pd.merge(final1,df_geo, on='Postal Code')
df_lat_long.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Using geopy library to get the latitude and longitude values of both the cities

### Latitude-Longitude values---New York City

In [30]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### Create a map of New York with neighborhoods superimposed on top.

In [31]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(nyc_df['Latitude'], nyc_df['Longitude'], nyc_df['Borough'], nyc_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

### Latitude-Longitude values---Toronto

In [32]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="CA_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [33]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_lat_long['Latitude'], df_lat_long['Longitude'], df_lat_long['Borough'], df_lat_long['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

#### Insights after observing the maps of two city:


Clearly the area of New York City is more than Toronto. Both Cites are located nearby bay.

## Using Foursquare API to explore neighborhoods two cities

In [34]:
CLIENT_ID = 'GOEAKDZWKLNFDCP55AHYG5MWOTUIUSPSSN5YMYLY4OPEFRGG' # your Foursquare ID
CLIENT_SECRET = 'VGRRZYDQJIQVSUFQRVT300ZXUWSDXYPXS3OZB120VSIYB4LY' # your Foursquare Secret
VERSION = '20191224'


### Exploring neighborhood of Toronto

Lets explore different venues in Toronto ___ We will be using FOURSQUARE API

In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [36]:
LIMIT=100
Toronto_venues = getNearbyVenues(names=df_lat_long['Neighbourhood'],
                                   latitudes=df_lat_long['Latitude'],
                                   longitudes=df_lat_long['Longitude']
                                  )


Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [188]:
Toronto_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",8,8,8,8,8,8
"Alderwood, Long Branch",10,10,10,10,10,10


In [38]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 277 uniques categories.


As we can see there are 277 unique venue categories in the neighborhood.


### Analysing each neighborhood

In [39]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [189]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond"

In [41]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [42]:
num_top_venues = 25

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Toronto_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
Toronto_neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    Toronto_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

Toronto_neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Bar,Thai Restaurant,Restaurant,Sushi Restaurant,Asian Restaurant,Burger Joint,Bakery,Salad Place,Cosmetics Shop,American Restaurant,Concert Hall,Gastropub,Vegetarian / Vegan Restaurant,Gym,Pizza Place,Hotel,Indian Restaurant,Department Store,Breakfast Spot,Plaza,Brazilian Restaurant,Ice Cream Shop
1,Agincourt,Latin American Restaurant,Lounge,Skating Rink,Breakfast Spot,Women's Store,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Empanada Restaurant,Eastern European Restaurant,Electronics Store,Diner,Ethiopian Restaurant,Event Space,Discount Store,Dim Sum Restaurant,Farmers Market,Coworking Space,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Coffee Shop,Playground,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Dumpling Restaurant,Dessert Shop,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Dim Sum Restaurant,Deli / Bodega,Department Store,Dance Studio,Curling Ice,Cupcake Shop,Cuban Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Fast Food Restaurant,Pharmacy,Pizza Place,Beer Store,Sandwich Place,Fried Chicken Joint,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dumpling Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Dessert Shop,Deli / Bodega,Department Store,Falafel Restaurant,Dance Studio,Curling Ice
4,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Dance Studio,Gym,Skating Rink,Pharmacy,Athletics & Sports,Sandwich Place,Pub,Dog Run,Diner,Discount Store,Drugstore,Doner Restaurant,Donut Shop,Dessert Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Dim Sum Restaurant,Curling Ice,Department Store,Deli / Bodega,Event Space


In [43]:
Toronto_neighborhoods_venues_sorted.shape

(100, 26)

In [44]:
#finding different cuisines in Toronto

list2=(np.unique(Toronto_neighborhoods_venues_sorted)).tolist()

key="Restaurant"
res = [i for i in list2 if key in i] 
len(res)
res

['American Restaurant',
 'Asian Restaurant',
 'Brazilian Restaurant',
 'Cajun / Creole Restaurant',
 'Caribbean Restaurant',
 'Chinese Restaurant',
 'Colombian Restaurant',
 'Comfort Food Restaurant',
 'Cuban Restaurant',
 'Dim Sum Restaurant',
 'Doner Restaurant',
 'Dumpling Restaurant',
 'Eastern European Restaurant',
 'Empanada Restaurant',
 'Ethiopian Restaurant',
 'Falafel Restaurant',
 'Fast Food Restaurant',
 'French Restaurant',
 'German Restaurant',
 'Greek Restaurant',
 'Hakka Restaurant',
 'Indian Restaurant',
 'Indonesian Restaurant',
 'Italian Restaurant',
 'Japanese Restaurant',
 'Korean Restaurant',
 'Latin American Restaurant',
 'Mediterranean Restaurant',
 'Mexican Restaurant',
 'Middle Eastern Restaurant',
 'New American Restaurant',
 'Portuguese Restaurant',
 'Ramen Restaurant',
 'Restaurant',
 'Seafood Restaurant',
 'South American Restaurant',
 'Southern / Soul Food Restaurant',
 'Sushi Restaurant',
 'Thai Restaurant',
 'Vegetarian / Vegan Restaurant',
 'Vietnamese

 Their are 44 different cuisines in the neighborhood of Toronto.


### Finding unique venues in Toronto

In [46]:
Toronto_venue_list=Toronto_neighborhoods_venues_sorted.values.tolist()

In [47]:
T_flat_list = []
for sublist in Toronto_venue_list:
    for item in sublist:
        T_flat_list.append(item)

In [ ]:
from collections import OrderedDict  
  
def group_list(lst): 
      
    res =  [(el, lst.count(el)) for el in lst] 
    return list(OrderedDict(res).items()) 
      
# Driver code 
T_b=group_list(T_flat_list)
T_b


In [50]:
# searching the list string elements with a substring.
key='Restaurant'
farms_count = [i for i in T_b if key in i] 
farms_count 

[('Restaurant', 31)]

### Exploring neighborhood of New York City

In [51]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
nyc_venues = getNearbyVenues(names=nyc_df['Neighborhood'],
                                   latitudes=nyc_df['Latitude'],
                                   longitudes=nyc_df['Longitude']
                                  )


In [53]:
print(nyc_venues.shape)
nyc_venues.head()

(10082, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Wakefield,40.894705,-73.847201,Shell,40.894187,-73.845862,Gas Station
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


In [190]:
nyc_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Annadale,10,10,10,10,10,10
Arden Heights,4,4,4,4,4,4
Arlington,5,5,5,5,5,5
Arrochar,20,20,20,20,20,20
Arverne,18,18,18,18,18,18


In [55]:
print('There are {} uniques categories.'.format(len(nyc_venues['Venue Category'].unique())))

There are 430 uniques categories.


In [56]:
# one hot encoding
nyc_onehot = pd.get_dummies(nyc_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
nyc_onehot['Neighborhood'] = nyc_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [nyc_onehot.columns[-1]] + list(nyc_onehot.columns[:-1])
nyc_onehot = nyc_onehot[fixed_columns]

nyc_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Christmas Market,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoo

In [191]:
nyc_grouped = nyc_onehot.groupby('Neighborhood').mean().reset_index()
nyc_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Christmas Market,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoo

In [58]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [192]:
num_top_venues = 25

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
nyc_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
nyc_neighborhoods_venues_sorted['Neighborhood'] = nyc_grouped['Neighborhood']

for ind in np.arange(nyc_grouped.shape[0]):
    nyc_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(nyc_grouped.iloc[ind, :], num_top_venues)

nyc_neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue
0,Annadale,American Restaurant,Pizza Place,Sports Bar,Dance Studio,Restaurant,Train Station,Park,Diner,Farm,Field,Fast Food Restaurant,Farmers Market,Women's Store,Falafel Restaurant,Factory,Eye Doctor,Exhibit,Filipino Restaurant,Fish & Chips Shop,Film Studio,Financial or Legal Service,Event Service,Fish Market,Fishing Store,Flea Market
1,Arden Heights,Pharmacy,Coffee Shop,Pizza Place,Playground,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court
2,Arlington,Deli / Bodega,American Restaurant,Bus Stop,Coffee Shop,Boat or Ferry,Women's Store,Financial or Legal Service,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fishing Store,Fish & Chips Shop,Fish Market,Event Space,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court
3,Arrochar,Italian Restaurant,Deli / Bodega,Bus Stop,Food Truck,Supermarket,Mediterranean Restaurant,Middle Eastern Restaurant,Liquor Store,Bagel Shop,Taco Place,Sandwich Place,Outdoors & Recreation,Athletics & Sports,Pizza Place,Polish Restaurant,Hotel,Cosmetics Shop,Fish Market,Fast Food Restaurant,Food & Drink Shop,Eye Doctor,Food,Factory,Falafel Restaurant,Farm
4,Arverne,Surf Spot,Metro Station,Sandwich Place,Donut Shop,Beach,Pizza Place,Playground,Coffee Shop,Bus Stop,Thai Restaurant,Bed & Breakfast,Wine Shop,Board Shop,Falafel Restaurant,Farm,Factory,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Women's Store,Film Studio,Financial or Legal Service,Fish & Chips Shop,Fish Market


In [60]:
nyc_venues_df=nyc_neighborhoods_venues_sorted.copy()

#### Getting insights on how many unique venues and their count are present in the neighborhoods of New York City

In [61]:
# Finding how many unique venues and there count in the neighborhoods of New York City 
ny_venue_list=nyc_neighborhoods_venues_sorted.values.tolist()

In [62]:
flat_list = []
for sublist in ny_venue_list:
    for item in sublist:
        flat_list.append(item)


In [ ]:

from collections import OrderedDict  
  
def group_list(lst): 
      
    res =  [(el, lst.count(el)) for el in lst] 
    return list(OrderedDict(res).items()) 
      
# Driver code 
b=group_list(flat_list)



In [193]:
# different venues and there count in New York City

nyc_unique_venues=pd.DataFrame(b)
nyc_unique_venues.head()

,0,1
0,Annadale,1
1,American Restaurant,70
2,Pizza Place,183
3,Sports Bar,11
4,Dance Studio,13


In [66]:
# To find out unique cusines in the city
list1=(np.unique(nyc_neighborhoods_venues_sorted)).tolist()

In [67]:
len(list1)

657

In [68]:

key='Restaurant'
ny_restaurants = [i for i in list1 if key in i] 
len(ny_restaurants)


67

There are total of 67 different types of cuisines in New York City and many other Fast Food Resturants. 

#### lets compare resturants in two cities.


In [94]:
# Different resturants in two cities

diff_res=list(set(ny_restaurants)-set(res))

In [95]:
len(diff_res)

29

In [96]:
Toronto_diff_res=list(set(res)-set(diff_res))
len(Toronto_diff_res)

41

In [97]:
nyc_diff_res=list(set(ny_restaurants)-set(diff_res))
len(nyc_diff_res)

38

In [98]:
#Toronto_unique_resturants
T_uniqueRest=list(set(Toronto_diff_res)-set(nyc_diff_res))
T_uniqueRest

['Portuguese Restaurant', 'Hakka Restaurant', 'Doner Restaurant']

In [187]:
#cross checking if unique resturants in toronto are present in NYC
print(ny_restaurants.count('Portuguese Restaurant'),ny_restaurants.count('Hakka Restaurant'),ny_restaurants.count('Hakka Restaurant'))

0 0 0


In [109]:
#unique restuarants in New york city
NYC_uniqueRest= list(set(ny_restaurants)&set(diff_res))
print(NYC_uniqueRest)



['Filipino Restaurant', 'African Restaurant', 'Taiwanese Restaurant', 'Peruvian Restaurant', 'Spanish Restaurant', 'Cantonese Restaurant', 'Turkish Restaurant', 'Egyptian Restaurant', 'Kosher Restaurant', 'Argentinian Restaurant', 'Halal Restaurant', 'Polish Restaurant', 'Hotpot Restaurant', 'English Restaurant', 'Pakistani Restaurant', 'Tapas Restaurant', 'Sri Lankan Restaurant', 'Shabu-Shabu Restaurant', 'Malay Restaurant', 'Szechuan Restaurant', 'Shanghai Restaurant', 'Russian Restaurant', 'Arepa Restaurant', 'Paella Restaurant', 'Kebab Restaurant', 'Hawaiian Restaurant', 'Tex-Mex Restaurant', 'Afghan Restaurant', 'Caucasian Restaurant']


## Cluster Neighborhoods

Let us cluster neighborhoods of both the cities using k-means clustering

### Clustering of Toronto neighborhood

Run *k*-means to cluster the neighborhood into 5 clusters.

In [133]:
Toronto_neighborhoods_venues_sorted.columns

Index(['Cluster Labels', 'Neighborhood', '1st Most Common Venue',
       '2nd Most Common Venue', '3rd Most Common Venue',
       '4th Most Common Venue', '5th Most Common Venue',
       '6th Most Common Venue', '7th Most Common Venue',
       '8th Most Common Venue', '9th Most Common Venue',
       '10th Most Common Venue', '11th Most Common Venue',
       '12th Most Common Venue', '13th Most Common Venue',
       '14th Most Common Venue', '15th Most Common Venue',
       '16th Most Common Venue', '17th Most Common Venue',
       '18th Most Common Venue', '19th Most Common Venue',
       '20th Most Common Venue', '21th Most Common Venue',
       '22th Most Common Venue', '23th Most Common Venue',
       '24th Most Common Venue', '25th Most Common Venue'],
      dtype='object')

In [69]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 0, 4, 4, 4, 4, 4, 4, 4], dtype=int32)

In [70]:
#Toronto_neighborhoods_venues_sorted.columns
#Toronto_merged.columns
#df_lat_long.columns

#### Let's create a new dataframe that includes the cluster as well as the top 25 venues for each neighborhood.

In [113]:
# add clustering labels
Toronto_neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = df_lat_long 

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(Toronto_neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood',how='inner')

Toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,3,Fast Food Restaurant,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Festival,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Dim Sum Restaurant,Women's Store,Department Store,Deli / Bodega,Dance Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,4,History Museum,Bar,Women's Store,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Dim Sum Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Diner,Dessert Shop,Festival,Department Store,Deli / Bodega,Dance Studio,Curling Ice,Cupcake Shop
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,4,Medical Center,Mexican Restaurant,Rental Car Location,Pizza Place,Breakfast Spot,Electronics Store,Intersection,Women's Store,Discount Store,Dog Run,Doner Restaurant,Drugstore,Donut Shop,Dim Sum Restaurant,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,Ethiopian Restaurant,Diner,Dessert Shop,Falafel Restaurant,Department Store,Deli / Bodega,Dance Studio,Curling Ice
3,M1G,Scarborough,Woburn,43.770992,-79.216917,4,Coffee Shop,Korean Restaurant,Women's Store,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Dim Sum Restaurant,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Diner,Dessert Shop,Festival,Department Store,Deli / Bodega,Dance Studio,Curling Ice,Cupcake Shop
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,4,Bakery,Bank,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Athletics & Sports,Gas Station,Thai Restaurant,Doner Restaurant,Discount Store,Dog Run,Women's Store,Diner,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Donut Shop,Department Store,Dim Sum Restaurant,Dessert Shop


Let's create a new dataframe that includes the cluster as well as the top 25 venues for each neighborhood.

In [114]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [168]:
def group_list(lst): 
      
    res =  [(el, lst.count(el)) for el in lst] 
    return list(OrderedDict(res).items()) 

In [169]:
t_c1=Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

In [173]:
t_c1_borough=t_c1.Borough.unique()
print(t_c1_borough)

['Scarborough' 'North York' 'East York' 'Central Toronto'
 'Downtown Toronto' 'York' 'Etobicoke']


In [176]:
t_venue_c1=t_c1.values.tolist()

flat_list_t_c1 = []
for sublist in t_venue_c1:
    for item in sublist:
        flat_list_t_c1.append(item)
        
t_c1_venues=group_list(flat_list_t_c1)
print(t_c1_venues)

[('Scarborough', 1), (0, 14), ('Park', 14), ('Coffee Shop', 2), ('Playground', 2), ("Women's Store", 8), ('Drugstore', 14), ('Diner', 14), ('Discount Store', 14), ('Dog Run', 14), ('Doner Restaurant', 14), ('Donut Shop', 14), ('Eastern European Restaurant', 14), ('Dumpling Restaurant', 14), ('Dessert Shop', 11), ('Electronics Store', 14), ('Empanada Restaurant', 14), ('Ethiopian Restaurant', 14), ('Event Space', 13), ('Falafel Restaurant', 12), ('Dim Sum Restaurant', 13), ('Deli / Bodega', 9), ('Department Store', 11), ('Dance Studio', 10), ('Curling Ice', 10), ('Cupcake Shop', 10), ('Cuban Restaurant', 9), ('North York', 6), ('Cafeteria', 1), ('Fast Food Restaurant', 6), ('Creperie', 3), ('Bank', 1), ('Bar', 1), ('Convenience Store', 2), ('Farmers Market', 6), ('Festival', 3), ('Coworking Space', 2), ('Colombian Restaurant', 4), ('Comfort Food Restaurant', 4), ('Comic Shop', 4), ('Concert Hall', 4), ('Food & Drink Shop', 1), ('Airport', 1), ('East York', 1), ('Central Toronto', 1), ('

In [177]:
t_c2=Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]
t_c2.Borough.unique()

array(['Etobicoke', 'North York'], dtype=object)

In [178]:
t_venue_c2=t_c2.values.tolist()

flat_list_t_c2 = []
for sublist in t_venue_c2:
    for item in sublist:
        flat_list_t_c2.append(item)
        
t_c2_venues=group_list(flat_list_t_c2)
print(t_c2_venues)

[('Etobicoke', 1), (1, 2), ('Locksmith', 1), ('Baseball Field', 2), ("Women's Store", 2), ('Eastern European Restaurant', 2), ('Dog Run', 2), ('Doner Restaurant', 2), ('Donut Shop', 2), ('Drugstore', 2), ('Dumpling Restaurant', 2), ('Electronics Store', 2), ('Diner', 2), ('Empanada Restaurant', 2), ('Ethiopian Restaurant', 2), ('Event Space', 2), ('Falafel Restaurant', 2), ('Farmers Market', 2), ('Discount Store', 2), ('Dim Sum Restaurant', 2), ('Festival', 1), ('Coworking Space', 2), ('Comfort Food Restaurant', 2), ('Comic Shop', 2), ('Concert Hall', 2), ('Construction & Landscaping', 2), ('Convenience Store', 2), ('North York', 1), ('Fast Food Restaurant', 1), ('Field', 1)]


In [180]:
t_c3=Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]
t_c3.Borough.unique()

array(['Central Toronto'], dtype=object)

In [183]:
t_venue_c3=t_c3.values.tolist()

flat_list_t_c3 = []
for sublist in t_venue_c3:
    for item in sublist:
        flat_list_t_c3.append(item)
        
t_c3_venues=group_list(flat_list_t_c3)
print(t_c3_venues)

[('Central Toronto', 1), (2, 1), ('Garden', 1), ('Dumpling Restaurant', 1), ('Diner', 1), ('Discount Store', 1), ('Dog Run', 1), ('Doner Restaurant', 1), ('Donut Shop', 1), ('Drugstore', 1), ('Eastern European Restaurant', 1), ('Dessert Shop', 1), ('Electronics Store', 1), ('Empanada Restaurant', 1), ('Ethiopian Restaurant', 1), ('Event Space', 1), ('Falafel Restaurant', 1), ('Farmers Market', 1), ('Dim Sum Restaurant', 1), ("Women's Store", 1), ('Festival', 1), ('Cosmetics Shop', 1), ('Colombian Restaurant', 1), ('Comfort Food Restaurant', 1), ('Comic Shop', 1), ('Concert Hall', 1), ('Construction & Landscaping', 1)]


In [181]:
t_c4=Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]
t_c4.Borough.unique()

array(['Scarborough'], dtype=object)

In [184]:
t_venue_c4=t_c4.values.tolist()

flat_list_t_c4 = []
for sublist in t_venue_c4:
    for item in sublist:
        flat_list_t_c4.append(item)
        
t_c4_venues=group_list(flat_list_t_c4)
print(t_c4_venues)

[('Scarborough', 1), (3, 1), ('Fast Food Restaurant', 1), ('Dumpling Restaurant', 1), ('Diner', 1), ('Discount Store', 1), ('Dog Run', 1), ('Doner Restaurant', 1), ('Donut Shop', 1), ('Drugstore', 1), ('Eastern European Restaurant', 1), ('Festival', 1), ('Electronics Store', 1), ('Empanada Restaurant', 1), ('Ethiopian Restaurant', 1), ('Event Space', 1), ('Falafel Restaurant', 1), ('Farmers Market', 1), ('Dim Sum Restaurant', 1), ("Women's Store", 1), ('Department Store', 1), ('Deli / Bodega', 1), ('Dance Studio', 1), ('Curling Ice', 1), ('Cupcake Shop', 1), ('Cuban Restaurant', 1), ('Creperie', 1)]


In [182]:
t_c5=Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]
t_c5.Borough.unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       "Queen's Park", 'Mississauga', 'Etobicoke'], dtype=object)

In [185]:
t_venue_c5=t_c5.values.tolist()

flat_list_t_c5 = []
for sublist in t_venue_c5:
    for item in sublist:
        flat_list_t_c5.append(item)
        
t_c5_venues=group_list(flat_list_t_c5)
print(t_c5_venues)

[('Scarborough', 14), (4, 83), ('History Museum', 3), ('Bar', 19), ("Women's Store", 30), ('Dumpling Restaurant', 51), ('Discount Store', 52), ('Dog Run', 49), ('Doner Restaurant', 54), ('Donut Shop', 53), ('Drugstore', 51), ('Eastern European Restaurant', 52), ('Dim Sum Restaurant', 42), ('Electronics Store', 55), ('Empanada Restaurant', 47), ('Ethiopian Restaurant', 48), ('Event Space', 39), ('Falafel Restaurant', 31), ('Farmers Market', 32), ('Diner', 61), ('Dessert Shop', 46), ('Festival', 10), ('Department Store', 36), ('Deli / Bodega', 34), ('Dance Studio', 27), ('Curling Ice', 22), ('Cupcake Shop', 15), ('Medical Center', 1), ('Mexican Restaurant', 11), ('Rental Car Location', 4), ('Pizza Place', 29), ('Breakfast Spot', 18), ('Intersection', 7), ('Coffee Shop', 49), ('Korean Restaurant', 3), ('Bakery', 24), ('Bank', 12), ('Hakka Restaurant', 1), ('Fried Chicken Joint', 13), ('Caribbean Restaurant', 6), ('Athletics & Sports', 8), ('Gas Station', 3), ('Thai Restaurant', 8), ('Play

### Clustering of New York City neighborhood

In [121]:
# set number of clusters
kclusters = 5

NYC_grouped_clustering = nyc_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(NYC_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 2, 2, 1, 1, 1, 1, 1, 1], dtype=int32)

In [122]:
# add clustering labels
nyc_neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

NYC_merged = nyc_df 

# merge nyc_grouped with nyc_data to add latitude/longitude for each neighborhood
NYC_merged = NYC_merged.join(nyc_neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood',how='inner')

NYC_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,2,Donut Shop,Pharmacy,Deli / Bodega,Sandwich Place,Dessert Shop,Laundromat,Ice Cream Shop,Gas Station,Farmers Market,Filipino Restaurant,Field,Fast Food Restaurant,Women's Store,Film Studio,Falafel Restaurant,Factory,Eye Doctor,Exhibit,Farm,Fish & Chips Shop,Financial or Legal Service,Event Service,Fish Market,Fishing Store,Flea Market
1,Bronx,Co-op City,40.874294,-73.829939,1,Bus Station,Ice Cream Shop,Fast Food Restaurant,Park,Chinese Restaurant,Bagel Shop,Grocery Store,Discount Store,Pizza Place,Pharmacy,Baseball Field,Restaurant,Mattress Store,Falafel Restaurant,Factory,Eye Doctor,Farm,Farmers Market,Women's Store,Filipino Restaurant,Field,Event Space,Film Studio,Financial or Legal Service,Fish & Chips Shop
2,Bronx,Eastchester,40.887556,-73.827806,1,Bus Station,Caribbean Restaurant,Deli / Bodega,Diner,Convenience Store,Chinese Restaurant,Fast Food Restaurant,Seafood Restaurant,Bakery,Bowling Alley,Metro Station,Platform,Donut Shop,Pizza Place,Cosmetics Shop,Bus Stop,Juice Bar,Flea Market,Food Court,Falafel Restaurant,Farm,Farmers Market,Food Stand,Field,Filipino Restaurant
3,Bronx,Fieldston,40.895437,-73.905643,1,River,Bus Station,Plaza,Women's Store,Film Studio,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Ethiopian Restaurant,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court
4,Bronx,Riverdale,40.890834,-73.912585,3,Park,Baseball Field,Plaza,Bus Station,Gym,Bank,Home Service,Playground,Food Truck,Distillery,Financial or Legal Service,Falafel Restaurant,Farm,Design Studio,Department Store,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Eye Doctor,Fish Market,Fishing Store,Flea Market


In [123]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(NYC_merged['Latitude'], NYC_merged['Longitude'], NYC_merged['Neighborhood'], NYC_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [149]:
def group_list(lst): 
      
    res =  [(el, lst.count(el)) for el in lst] 
    return list(OrderedDict(res).items()) 

In [124]:
nyc_c1=NYC_merged.loc[NYC_merged['Cluster Labels'] == 0, NYC_merged.columns[[1] + list(range(5, NYC_merged.shape[1]))]]

In [142]:
nyc_c1_neigh=nyc_c1.Neighborhood.unique()
print(nyc_c1_neigh)


['Mill Island']


In [155]:
ny_venue_c1=nyc_c1.values.tolist()


In [151]:
ny_venue_c1=nyc_c1.values.tolist()

flat_list_c1 = []
for sublist in ny_venue_c1:
    for item in sublist:
        flat_list_c1.append(item)
        
c1_venues=group_list(flat_list_c1)
print(c1_venues)

In [154]:
c1_venues=group_list(flat_list_c1)
print(c1_venues)

[('Mill Island', 1), ('Pool', 1), ('Financial or Legal Service', 1), ('Event Space', 1), ('Exhibit', 1), ('Eye Doctor', 1), ('Factory', 1), ('Falafel Restaurant', 1), ('Farm', 1), ('Farmers Market', 1), ('Fast Food Restaurant', 1), ('Field', 1), ('Filipino Restaurant', 1), ('Film Studio', 1), ("Women's Store", 1), ('Ethiopian Restaurant', 1), ('Fish & Chips Shop', 1), ('Fish Market', 1), ('Fishing Store', 1), ('Flea Market', 1), ('Flower Shop', 1), ('Food', 1), ('Food & Drink Shop', 1), ('Food Court', 1), ('Food Stand', 1), ('Food Truck', 1)]


In [126]:
nyc_c2=NYC_merged.loc[NYC_merged['Cluster Labels'] == 1, NYC_merged.columns[[1] + list(range(5, NYC_merged.shape[1]))]]

In [159]:
nyc_c2_neigh=nyc_c2.Neighborhood.unique()
print(len(nyc_c2_neigh))

233


In [160]:
ny_venue_c2=nyc_c2.values.tolist()

flat_list_c2 = []
for sublist in ny_venue_c2:
    for item in sublist:
        flat_list_c2.append(item)
        
c2_venues=group_list(flat_list_c2)
print(c2_venues)

[('Co-op City', 1), ('Bus Station', 49), ('Ice Cream Shop', 70), ('Fast Food Restaurant', 127), ('Park', 80), ('Chinese Restaurant', 103), ('Bagel Shop', 52), ('Grocery Store', 80), ('Discount Store', 41), ('Pizza Place', 164), ('Pharmacy', 94), ('Baseball Field', 12), ('Restaurant', 41), ('Mattress Store', 8), ('Falafel Restaurant', 110), ('Factory', 93), ('Eye Doctor', 73), ('Farm', 110), ('Farmers Market', 110), ("Women's Store", 61), ('Filipino Restaurant', 86), ('Field', 89), ('Event Space', 43), ('Film Studio', 77), ('Financial or Legal Service', 68), ('Fish & Chips Shop', 65), ('Eastchester', 1), ('Caribbean Restaurant', 42), ('Deli / Bodega', 95), ('Diner', 50), ('Convenience Store', 29), ('Seafood Restaurant', 41), ('Bakery', 95), ('Bowling Alley', 6), ('Metro Station', 19), ('Platform', 1), ('Donut Shop', 89), ('Cosmetics Shop', 41), ('Bus Stop', 29), ('Juice Bar', 16), ('Flea Market', 44), ('Food Court', 36), ('Food Stand', 16), ('Fieldston', 1), ('River', 2), ('Plaza', 21),

In [ ]:
nyc_c3=NYC_merged.loc[NYC_merged['Cluster Labels'] == 2, NYC_merged.columns[[1] + list(range(5, NYC_merged.shape[1]))]]


In [162]:
nyc_c3_neigh=nyc_c3.Neighborhood.unique()
print(len(nyc_c3_neigh))

40


In [161]:
ny_venue_c3=nyc_c3.values.tolist()

flat_list_c3 = []
for sublist in ny_venue_c3:
    for item in sublist:
        flat_list_c3.append(item)
        
c3_venues=group_list(flat_list_c3)
print(c3_venues)

[('Wakefield', 1), ('Donut Shop', 16), ('Pharmacy', 7), ('Deli / Bodega', 40), ('Sandwich Place', 9), ('Dessert Shop', 6), ('Laundromat', 1), ('Ice Cream Shop', 10), ('Gas Station', 3), ('Farmers Market', 33), ('Filipino Restaurant', 28), ('Field', 29), ('Fast Food Restaurant', 34), ("Women's Store", 21), ('Film Studio', 26), ('Falafel Restaurant', 34), ('Factory', 34), ('Eye Doctor', 31), ('Exhibit', 15), ('Farm', 36), ('Fish & Chips Shop', 24), ('Financial or Legal Service', 25), ('Event Service', 9), ('Fish Market', 26), ('Fishing Store', 26), ('Flea Market', 24), ('Longwood', 1), ('Grocery Store', 7), ('Diner', 6), ('Latin American Restaurant', 3), ('Flower Shop', 18), ('Food', 21), ('Throgs Neck', 1), ('Coffee Shop', 11), ('Asian Restaurant', 2), ('Sports Bar', 3), ('Chinese Restaurant', 14), ('Bar', 5), ('Pizza Place', 19), ('Italian Restaurant', 16), ('Juice Bar', 1), ('American Restaurant', 5), ('Dim Sum Restaurant', 2), ('Design Studio', 1), ('Van Nest', 1), ('Bus Station', 7)

In [ ]:
nyc_c4=NYC_merged.loc[NYC_merged['Cluster Labels'] == 3, NYC_merged.columns[[1] + list(range(5, NYC_merged.shape[1]))]]


In [163]:
nyc_c4_neigh=nyc_c4.Neighborhood.unique()
print(len(nyc_c4_neigh))

20


In [164]:
ny_venue_c4=nyc_c4.values.tolist()

flat_list_c4 = []
for sublist in ny_venue_c4:
    for item in sublist:
        flat_list_c4.append(item)
        
c4_venues=group_list(flat_list_c4)
print(c4_venues)

[('Riverdale', 1), ('Park', 10), ('Baseball Field', 2), ('Plaza', 1), ('Bus Station', 3), ('Gym', 2), ('Bank', 2), ('Home Service', 1), ('Playground', 5), ('Food Truck', 7), ('Distillery', 1), ('Financial or Legal Service', 20), ('Falafel Restaurant', 20), ('Farm', 20), ('Design Studio', 3), ('Department Store', 2), ('Farmers Market', 20), ('Fast Food Restaurant', 20), ('Field', 20), ('Filipino Restaurant', 20), ('Film Studio', 20), ('Fish & Chips Shop', 20), ('Eye Doctor', 17), ('Fish Market', 19), ('Fishing Store', 20), ('Flea Market', 19), ('Clason Point', 1), ('Bus Stop', 13), ('Grocery Store', 1), ('Moving Target', 1), ('Pool', 2), ('Boat or Ferry', 1), ('South American Restaurant', 1), ("Women's Store", 14), ('Factory', 18), ('Exhibit', 13), ('Flower Shop', 17), ('Food', 16), ('Food & Drink Shop', 15), ('Spuyten Duyvil', 1), ('Intersection', 3), ('Pharmacy', 1), ('Bus Line', 1), ('Thai Restaurant', 1), ('Tennis Stadium', 1), ('Event Service', 8), ('South Ozone Park', 1), ('Bar', 

In [130]:
nyc_c5=NYC_merged.loc[NYC_merged['Cluster Labels'] == 4, NYC_merged.columns[[1] + list(range(5, NYC_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue
259,Howland Hook,Boat or Ferry,Women's Store,Fish & Chips Shop,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish Market,Frozen Yogurt Shop,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain


In [146]:
nyc_c5_neigh=nyc_c5.Neighborhood.unique()
print(nyc_c5_neigh)

['Howland Hook']


In [165]:
ny_venue_c5=nyc_c5.values.tolist()

flat_list_c5 = []
for sublist in ny_venue_c5:
    for item in sublist:
        flat_list_c5.append(item)
        
c5_venues=group_list(flat_list_c5)
print(c5_venues)

[('Howland Hook', 1), ('Boat or Ferry', 1), ("Women's Store", 1), ('Fish & Chips Shop', 1), ('Exhibit', 1), ('Eye Doctor', 1), ('Factory', 1), ('Falafel Restaurant', 1), ('Farm', 1), ('Farmers Market', 1), ('Fast Food Restaurant', 1), ('Field', 1), ('Filipino Restaurant', 1), ('Film Studio', 1), ('Financial or Legal Service', 1), ('Fish Market', 1), ('Frozen Yogurt Shop', 1), ('Fishing Store', 1), ('Flea Market', 1), ('Flower Shop', 1), ('Food', 1), ('Food & Drink Shop', 1), ('Food Court', 1), ('Food Stand', 1), ('Food Truck', 1), ('Fountain', 1)]
